# Day 11: Corporate Policy

[*Advent of Code 2015 day 11*](https://adventofcode.com/2015/day/11) and [*solution megathread*](https://redd.it/3wbzyv)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/11/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F11%2Fcode.ipynb)

In [1]:
%load_ext nb_mypy

Version 1.0.4


In [2]:
%nb_mypy On

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

3:1: E402 module level import not at top of file
6:20: E225 missing whitespace around operator


Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [6]:
from IPython.display import HTML


HTML(downloaded['part1'])

## Comments

It's 2021-12-05, and for some reason I put off solving this one and those after - prefering to fill the repository with more problems. Now it is time though.

_Update 2022-11-19:_ Funny how some problems just feel too tough (and boring?), so I didn't solve this until now... Anyway, the classes (to simplify operations) may be introducing unreasonable costs, so the _test_ problems take frustratingly long to verify.

In [7]:
from IPython.display import display

inputdata = downloaded['input']
display(f'{inputdata =}')

"inputdata ='vzbxkghb'"

In [8]:
from typing import List
from itertools import chain


class Password:
    def __init__(self, os: List[int]):
        # Note that internally, characters are stored in a list
        # least significant position first
        self.os = os

    def __str__(self) -> str:
        return ''.join([chr(o) for o in self.os][::-1])

    def __repr__(self) -> str:
        return f"'{str(self)}'"

    @staticmethod
    def from_string(cs_be: str):
        return Password([ord(c) for c in cs_be[::-1]])

    # Passwords must include one increasing straight of at least three letters,
    #   like abc, bcd, cde, and so on, up to xyz. They cannot skip letters; abd
    #   doesn't count.
    def rule_3straight(self) -> bool:
        # Note that we go through characters least significant position
        # first
        for trigram in zip(self.os[:-2], self.os[1:-1], self.os[2:]):
            incremented = (trigram[0], trigram[1] + 1, trigram[2] + 2)
            if (
                    not any(o > ord('z') for o in incremented)
                    and len(set(incremented)) <= 1):
                return True
        return False

    # Passwords may not contain the letters i, o, or l, as these letters can
    #   be mistaken for other characters and are therefore confusing.
    # Instantiate static objects, since I suspect this comparison otherwise
    # is unreasonably costly
    def rule_noconfusing(self) -> bool:
        return not any(ord(confusing) in self.os for confusing in 'iol')

    # Passwords must contain at least two different, non-overlapping pairs of
    #   letters, like aa, bb, or zz.
    def rule_twopairs(self) -> bool:
        bigrams = zip(self.os[:-1], self.os[1:])
        bg_pairs = list(chain([False],
                              map(lambda bg: bg[0] == bg[1], bigrams),
                              [False]))
        overlapping = zip(bg_pairs[:-1], bg_pairs[1:])
        non_overlapping = map(lambda ol: any(ol), overlapping)
        return sum(non_overlapping) >= 4

    def is_valid(self) -> bool:
        # Evaluate these lazily in the order of fastest to most complex
        return all(f() for f in
                   (self.rule_noconfusing,
                    self.rule_twopairs,
                    self.rule_3straight))

    def next(self):
        next_os = self.os.copy()
        # Increment starting from least significant until we don't carry
        for i in range(len(next_os)):
            next_os[i] += 1
            if next_os[i] > ord('z'):
                next_os[i] = ord('a')
            else:
                break
        return Password(next_os)

    def next_valid(self):
        next_pass = self.next()
        while not next_pass.is_valid():
            next_pass = next_pass.next()
        return next_pass

In [9]:
if False:
    from cProfile import run
    from pstats import SortKey, Stats
    abcdefgh = Password.from_string("abcdefgh")
    run('abcdefgh.next_valid()', sort=SortKey.CUMULATIVE)

It was kind of interesting, as mentioned the class for each character cost a lot, but with some more effort I could do all from the `Password` class, as well as switched to just keeping a list of integers rather than a string. Perhaps converting between them still cost some, but now I do think the `zip(os[:-1], os[1:])` ruins the flow most. Plus I finally got to check out the profiler:

```
>>> run('abcdefgh.next_valid()', sort=SortKey.CUMULATIVE)
         598669 function calls in 0.160 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.180    0.180 {built-in method builtins.exec}
        1    0.000    0.000    0.180    0.180 <string>:1(<module>)
        1    0.009    0.009    0.180    0.180 <stdin>:58(next_valid)
    17413    0.014    0.000    0.143    0.000 <stdin>:42(is_valid)
    17413    0.005    0.000    0.127    0.000 {built-in method builtins.all}
    46849    0.013    0.000    0.124    0.000 <stdin>:44(<genexpr>)
    12022    0.029    0.000    0.077    0.000 <stdin>:34(rule_twopairs)
    12022    0.014    0.000    0.038    0.000 {built-in method builtins.sum}
   113594    0.016    0.000    0.034    0.000 {built-in method builtins.any}
    17413    0.009    0.000    0.034    0.000 <stdin>:30(rule_noconfusing)
    96176    0.015    0.000    0.024    0.000 <stdin>:40(<lambda>)
    63967    0.015    0.000    0.018    0.000 <stdin>:31(<genexpr>)
    84154    0.010    0.000    0.010    0.000 <stdin>:37(<lambda>)
    65376    0.005    0.000    0.005    0.000 {built-in method builtins.ord}
    17413    0.003    0.000    0.003    0.000 <stdin>:2(__init__)
    17418    0.002    0.000    0.002    0.000 {built-in method builtins.len}
    17413    0.002    0.000    0.002    0.000 {method 'copy' of 'list' objects}
        1    0.000    0.000    0.000    0.000 <stdin>:48(next)
        1    0.000    0.000    0.000    0.000 <stdin>:16(rule_3straight)
       20    0.000    0.000    0.000    0.000 <stdin>:22(<genexpr>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
```

In [10]:
%pycodestyle_off
hijklmmn = Password.from_string("hijklmmn")
assert hijklmmn.rule_3straight() == True, "hijklmmn meets the first requirement " + \
    "(because it contains the straight hij)"
assert hijklmmn.rule_noconfusing() == False, "...but fails the second requirement " + \
    "requirement (because it contains i and l)"
abbceffg = Password.from_string("abbceffg")
assert abbceffg.rule_twopairs() == True, "abbceffg meets the third requirement " + \
    "(because it repeats bb and ff)"
assert abbceffg.rule_3straight() == False, "...but fails the first requirement"
abbcegjk = Password.from_string("abbcegjk")
assert abbcegjk.rule_twopairs() == False, "abbcegjk fails the third requirement, " + \
    "because it only has one double letter (bb)"
abcdefgh = Password.from_string("abcdefgh")
assert str(abcdefgh.next_valid()) == "abcdffaa", "The next password after " + \
    "abcdefgh is abcdffaa"
ghijklmn = Password.from_string("ghijklmn")
next = ghijklmn.next_valid() 
assert str(next) == "ghjaabcc", "The next password after " + \
    "ghijklmn is ghjaabcc, because you eventually skip all the passwords that " + \
    "start with ghi..., since i is not allowed"
%pycodestyle_on

In [11]:
def my_part1_solution(data: str) -> str:
    return str(Password.from_string(data).next_valid())

In [12]:
display(my_part1_solution(inputdata))

'vzbxxyzz'

In [13]:
HTML(downloaded['part1_footer'])

## Part Two

In [14]:
HTML(downloaded['part2'])

In [15]:
def my_part2_solution(data: str) -> str:
    return str(Password.from_string(data).next_valid().next_valid())

In [16]:
display(my_part2_solution(inputdata))

'vzcaabcc'

In [17]:
HTML(downloaded['part2_footer'])